# Authorship verification experiment using the Kaggle Blog Corpus with Claude

In [1]:
import os 
import sys
sys.path.append(os.path.abspath('..'))

In [2]:
import pandas as pd
import numpy as np

In [3]:
from anthropic import Anthropic
from credentials import get_credentials_claude

API_KEY = get_credentials_claude()

client = Anthropic(api_key=API_KEY)


In [4]:
# Load the training and test sets
train_path = "./data/authorship_verification_kaggle_data_2_train.pkl"
test_path = "./data/authorship_verification_kaggle_data_2_test.pkl"

df_train = pd.read_pickle(train_path)
df_test = pd.read_pickle(test_path)

In [39]:
#Create training string

# Set random seed for reproducibility
np.random.seed(42)

# Get unique author IDs
author_ids = df_train['id'].unique()

# Dictionary to store concatenated texts
author_samples = {}

for author_id in author_ids:
    # Sample 30 texts for this author
    sample_texts = df_train[df_train['id'] == author_id]['text'].sample(n=35, random_state=42)
    # Concatenate into a single string (separated by newlines or spaces)
    concatenated = "\n\n".join(sample_texts)
    author_samples[author_id] = concatenated




In [22]:
# Inspect the data

for id in author_ids:
    print(f"Author: {id}")
    print(author_samples[id][:500])


Author: 734562
           nah, he's still has that huge ass poster in his bedroom.   and 'orthographical'?  wow, big word.          

           when downloading porn just isn't enough anymore:  urlLink imaginarygirlfriends.com          

           33% MORE!         

           You must all think of me on Sunday, October 27th. It is my 29th birthday. I will probably be unable to attend the Blog-a-bration but will keep you, um, posted.         

           Bless your heart, Mir...first Clark, now Edwards...no
Author: 449628
        Letters of Recommendation   Letters of recommendation are something that you could put in a professional portfolio, for example.  They are often requested in academic settings.  I found some links that hopefully will help  Michael Ernst, an Assistant Professor at MIT, offers his advice:   urlLink Requesting a letter of recommendation   urlLink Writing a letter of recommendation   Monster.com also has some advice:  urlLink Writing a Letter of Recommendation 

In [10]:
# Pick a random text from the test sample and get its author id
test_text = df_test['text'].sample(n=1, random_state=42).values[0]
test_author_id = df_test[df_test['text'] == test_text]['id'].values[0]


In [11]:
print(test_text)
print(test_author_id)

           Well, you could practice both at the same time...... Provided that the one you are dating is the one you are married to as well.          
734562


In [43]:
# Create the template prompt

prompt = f"""
You are an expert in linguistic analysis and authorship attribution. Your task is to analyze authorship--i.e. idiolectal--markers that the \
texts reveal about their authors.


You will be provided with:
1. Training samples from 2 different authors, identified only by an unique id number (734562 and 449628);
2. A new text whose author you need to identify between those two authors

Guideline for analysis:
- Just use state of the art techniques to attribute authorship

Author's id 734562 samples:
{author_samples[author_ids[0]]}

Author's id 449628 samples:
{author_samples[author_ids[1]]}

New text to analyze:
{test_text}

Give as your answer only the author's id number and nothing else, like so: NNNNNN. Do not include any other text.

"""

In [44]:
# Create a completion function that will be used to query the model
def get_completion(prompt: str, max_tokens=5000) -> str:
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        )
    return response.content[0].text

In [45]:
first_prediction = get_completion(prompt)
print(first_prediction)

734562


In [20]:
# Check if the prediction is accurate
# Get the actual author of the random tweet
actual_author = str(test_author_id).lower()

# Get the predicted author from the model's response
predicted_author = first_prediction.strip().lower()

# Check if the prediction matches the actual author
is_correct = predicted_author == actual_author

print(f"Actual author: {actual_author}")
print(f"Predicted author: {predicted_author}")
print(f"Prediction is correct: {is_correct}")

# Calculate accuracy (1 for correct, 0 for incorrect)
accuracy = 1 if is_correct else 0
print(f"Accuracy: {accuracy}")



Actual author: 734562
Predicted author: 734562
Prediction is correct: True
Accuracy: 1


In [24]:
# Sample 100 test blog posts for evaluation
n_samples = 100
sampled_posts = df_test.sample(n=n_samples, random_state=42).reset_index(drop=True)
print(f'Created sample dataset with {len(sampled_posts)} blog posts')
print('Sample distribution:')
print(sampled_posts['id'].value_counts())

Created sample dataset with 100 blog posts
Sample distribution:
id
449628    60
734562    40
Name: count, dtype: int64


In [25]:
def normalize_author_id(author_id):
    return str(author_id).strip()

In [26]:
import time

batch_size = 10
total_batches = len(sampled_posts) // batch_size + (1 if len(sampled_posts) % batch_size > 0 else 0)

results = []  # (idx, text, actual_author, predicted_author)
errors = []
print(f'Starting processing of {len(sampled_posts)} blog posts in {total_batches} batches')

for batch in range(total_batches):
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, len(sampled_posts))
    batch_errors = 0
    for idx in range(start_idx, end_idx):
        row = sampled_posts.iloc[idx]
        blog_text = row['text']
        actual_author = normalize_author_id(row['id'])
        prompt = f"""
You are an expert in linguistic analysis and authorship attribution. Your task is to analyze authorship--i.e. idiolectal--markers that the texts reveal about their authors.

You will be provided with:
1. Training samples from 2 different authors, identified only by an unique id number (734562 and 449628);
2. A new text whose author you need to identify between those two authors

Guideline for analysis:
- Just use state of the art techniques to attribute authorship

Author's id 734562 samples:
{author_samples[author_ids[0]]}

Author's id 449628 samples:
{author_samples[author_ids[1]]}

New text to analyze:
{blog_text}

Give as your answer only the author's id number and nothing else, like so: NNNNNN. Do not include any other text.
"""
        max_retries = 3
        retry_delay = 120
        for retry in range(max_retries):
            try:
                raw_prediction = get_completion(prompt).strip()
                prediction = normalize_author_id(raw_prediction)
                if prediction in [normalize_author_id(author_ids[0]), normalize_author_id(author_ids[1])]:
                    results.append((idx, blog_text, actual_author, prediction))
                else:
                    batch_errors += 1
                    errors.append((idx, f'Invalid prediction: {raw_prediction}'))
                time.sleep(10)
                break
            except Exception as e:
                if retry < max_retries - 1:
                    time.sleep(retry_delay)
                else:
                    batch_errors += 1
                    errors.append((idx, str(e)))
                    break
    print(f'Completed batch {batch + 1}/{total_batches} (posts {start_idx}-{end_idx-1})')
    if batch_errors > 0:
        print(f'  ⚠️  {batch_errors} errors in this batch')
    if batch < total_batches - 1:
        print('  Taking a 30 second break...')
        time.sleep(30)

print('\nProcessing complete!')
print(f'Successfully processed: {len(results)} blog posts')
if errors:
    print(f'Total errors: {len(errors)}')

Starting processing of 100 blog posts in 10 batches
Completed batch 1/10 (posts 0-9)
  ⚠️  9 errors in this batch
  Taking a 30 second break...
Completed batch 2/10 (posts 10-19)
  ⚠️  7 errors in this batch
  Taking a 30 second break...
Completed batch 3/10 (posts 20-29)
  ⚠️  7 errors in this batch
  Taking a 30 second break...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

if results:
    predicted_authors = [r[3] for r in results]
    actual_authors = [r[2] for r in results]
    correct = sum(p == a for p, a in zip(predicted_authors, actual_authors))
    final_accuracy = correct / len(predicted_authors)
    print(f'Overall accuracy: {final_accuracy:.2%} ({correct}/{len(predicted_authors)})')
    # Per-author accuracy
    for author in [normalize_author_id(author_ids[0]), normalize_author_id(author_ids[1])]:
        author_cases = [(p == author) for _, _, a, p in results if a == author]
        if author_cases:
            author_accuracy = sum(author_cases) / len(author_cases)
            print(f'Accuracy for {author}: {author_accuracy:.2%} ({sum(author_cases)}/{len(author_cases)})')
    # Confusion matrix
    valid_results = [(a, p) for _, _, a, p in results if p in [normalize_author_id(author_ids[0]), normalize_author_id(author_ids[1])]]
    if valid_results:
        y_true = [r[0] for r in valid_results]
        y_pred = [r[1] for r in valid_results]
        cm = confusion_matrix(y_true, y_pred, labels=[normalize_author_id(author_ids[0]), normalize_author_id(author_ids[1])])
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=[str(author_ids[0]), str(author_ids[1])], yticklabels=[str(author_ids[0]), str(author_ids[1])])
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()
    # Qualitative examples
    results_df = pd.DataFrame(results, columns=['idx', 'text', 'actual_author', 'predicted_author'])
    correct_cases = results_df[results_df['actual_author'] == results_df['predicted_author']]
    incorrect_cases = results_df[results_df['actual_author'] != results_df['predicted_author']]
    n_samples = 10
    sampled_correct = correct_cases.sample(n=min(n_samples, len(correct_cases)), random_state=42)
    sampled_incorrect = incorrect_cases.sample(n=min(n_samples, len(incorrect_cases)), random_state=42)
    print('SOME CORRECTLY CLASSIFIED EXAMPLES:\\n')
    for _, row in sampled_correct.iterrows():
        print(f'Blog: {row[\"text\"][:100]}...\\nActual: {row[\"actual_author\"]} | Predicted: {row[\"predicted_author\"]}\\n{\"-\"*60}')
    print('\\nSOME INCORRECTLY CLASSIFIED EXAMPLES:\\n')
    for _, row in sampled_incorrect.iterrows():
        print(f'Blog: {row[\"text\"][:100]}...\\nActual: {row[\"actual_author\"]} | Predicted: {row[\"predicted_author\"]}\\n{\"-\"*60}')